# Auxilary testing notebook for A2
Checking if the solution is correct by creating a sample case from datase.json

In [ ]:
import json
with open('/data/PanoHeadData/single_view_hq/dataset_v2_balanced_nohat.json', 'r') as f:
    all_data = json.load(f)

sample_items = {}
visited_views = []
for image_name, image_meta in all_data.items():
    cur_view = image_meta['view']
    if cur_view in visited_views:
        continue
    visited_views.append(cur_view)
    sample_items[image_name] = image_meta
    if len(visited_views) == 4:
        break
print(json.dumps(sample_items, indent=4))

In [11]:
import numpy as np
# from dpmain import DatProcV1
# dp = DatProcV1()
dc = {'00000/img00000001.jpg': {'data_source': 'FFHQ', 'camera': [0.9807107448577881, -0.014989661052823067, 0.19418708980083466, -0.5243051648139954, 0.008136472664773464, -0.9863376021385193, -0.16408364474773407, 0.4430258572101593, 0.19552138447761536, 0.16432251036167145, -0.9671899080276489, 2.611412763595581, 0.0, 0.0, 0.0, 1.0, 4.2647, 0.0, 0.5, 0.0, 4.2647, 0.5, 0.0, 0.0, 1.0], 'hpose': [-11.207698822021484, 0.7423303127288818, -9.562204360961914], 'align_box': [106.41131664853101, 99.16866158868335, 803.3775843307943, 803.3775843307943], 'align_quad': [[106.46320729053319, 99.29797589091405], [106.46313928182808, 903.0629760609359], [910.227965179543, 903.0630440696409], [910.2281011969533, 99.29797589091405]], 'view': 'front', 'head_image_path': 'head_images/27658_00.png', 'head_parsing_path': 'head_parsing/27658_00.png', 'align_image_path': 'align_images/27658_00.png', 'align_parsing_path': 'align_parsing/27658_00.png', 'camera_scoord': [78.64740352231533, 80.5564151920759, 2.7001193377103423, -0.5243051648139954, 0.4430258572101593, 2.611412763595581], 'svd_score': 0.5187521027959513, 'laplacian_score': 342.23823780679396, 'iv2b_ratio': 0.08324265505984763, 'head_region_thresh': 0.8025390495600516, 'dup_num': 1}}

def get_cam_coords(c2w):
    # Copied from datproc_v1.py
    # World Coordinate System: x(right), y(up), z(forward)
    T = c2w[:3, 3]
    x, y, z = T
    r = np.sqrt(x**2+y**2+z**2)
    # theta = np.rad2deg(np.arctan(np.sqrt(x**2+z**2)/y))
    theta = np.rad2deg(np.arctan2(x, z))
    if theta >= -90 and theta <= 90:
        theta += 90
    elif theta >= -180 and theta < -90:
        theta += 90
    elif theta > 90 and theta <= 180:
        theta -= 270
    else:
        raise ValueError('theta out of range')
    # phi = np.rad2deg(np.arctan(z/x))+180
    phi = np.rad2deg(np.arccos(y/r))
    return [theta, phi, r, x, y, z]  # [:3] sperical cood, [3:] cartesian cood

def _is_valid_cross_180(image_degree, min_angle, max_angle):
    # in this case min_angle is larger than max_angle. e.g. for 'left', min_angle = 135, max_angle = -135
    assert min_angle > max_angle and image_degree <= 180 and image_degree >= -180
    return image_degree <= max_angle or image_degree >= min_angle

def _is_valid(image_degree, min_angle, max_angle):
    return min_angle <= image_degree and image_degree <= max_angle


In [14]:
partition = [135, -135]
is_valid = _is_valid_cross_180 if partition[0] > partition[1] else _is_valid
new_data = {}
for image_name, image_meta in sample_items.items():
    cur_cam = image_meta['camera']
    cur_TMatrix = np.array(cur_cam[:16]).reshape(4, 4)
    cur_cam_scoord = get_cam_coords(cur_TMatrix)
    theta = cur_cam_scoord[0]
    if is_valid(theta, *partition):
        new_data[image_name] = image_meta
print(json.dumps(new_data, indent=4))

{
    "00094/img00094534.jpg": {
        "data_source": "LPFF",
        "camera": [
            0.6634543538093567,
            -0.19513431191444397,
            -0.7218035459518433,
            1.9488695859909058,
            -0.03121918812394142,
            -0.9712330102920532,
            0.2364508956670761,
            -0.6384174227714539,
            -0.7475708723068237,
            -0.13651904463768005,
            -0.6504607200622559,
            1.7562439441680908,
            0.0,
            0.0,
            0.0,
            1.0,
            4.2647,
            0.0,
            0.5,
            0.0,
            4.2647,
            0.5,
            0.0,
            0.0,
            1.0
        ],
        "hpose": [
            48.35384750366211,
            -2.5738792419433594,
            11.721717834472656
        ],
        "align_box": [
            140.70229007633588,
            183.6946564885496,
            703.5114503816793,
            703.5114503816793
        ],
 